In [1]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout, GRU
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 
from keras.utils import np_utils

import pandas as pd
import numpy as np
import string, os 

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#path = "/content/drive/MyDrive/subject_lines_final.csv"
path = "/content/drive/MyDrive/tweet_dataset.csv"
df = pd.read_csv(path)
df

,textID,sentiment,author,text,old_text,aux_id,new_sentiment,selected_text
0,1956967341,empty,xoshayzers,i know i was listenin to bad habit earlier a...,@tiffanylue i know i was listenin to bad habi...,p1000000000,NaN,NaN
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...,Layin n bed with a headache ughhhh...waitin o...,c811396dc2,negative,headache
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...,Funeral ceremony...gloomy friday...,9063631ab1,negative,gloomy
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!,wants to hang out with friends SOON!,2a815f151d,positive,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,We want to trade with someone who has Houston...,@dannycastillo We want to trade with someone w...,82565a56d3,neutral,We want to trade with someone who has Houston ...
...,...,...,...,...,...,...,...,...
39995,1753918954,neutral,showMe_Heaven,NaN,@JohnLloydTaylor,p1000008985,neutral,NaN
39996,1753919001,love,drapeaux,Happy Mothers Day All my love,Happy Mothers Day All my love,0b62ea4f2d,positive,Happy
39997,1753919005,love,JenniRox,Happy Mother`s Day to all the mommies out ther...,Happy Mother's Day to all the mommies out ther...,1adaa3519d,positive,Happy Mother`s Day
39998,1753919043,happiness,ipdaman1,WASSUP BEAUTIFUL!!! FOLLOW ME!! PEEP OUT MY ...,@niariley WASSUP BEAUTIFUL!!! FOLLOW ME!! PEE...,d63253be9a,neutral,WASSUP BEAUTIFUL!!! FOLLOW ME!! PEEP OUT MY N...


In [4]:
df_sub = df["text"] #Remove source, serial number columns
#df_sub = df["subjects"]
df_sub = df_sub.dropna()
df_sub = df_sub.iloc[:15000]
df_sub

0         i know  i was listenin to bad habit earlier a...
1        Layin n bed with a headache  ughhhh...waitin o...
2                      Funeral ceremony...gloomy friday...
3                     wants to hang out with friends SOON!
4         We want to trade with someone who has Houston...
                               ...                        
15020         haha wtf is that! You don`t want me to leave
15021    Computer pissing me off. 4 gig ram 2.5ghz Dual...
15022    IN PAIN. my big toe got stamped on during the ...
15023    Latest: Saw Animal Collective in Oakland and t...
15024    I h8 this weather!! It`s the be gloomy & rainy...
Name: text, Length: 15000, dtype: object

In [5]:
len(df_sub)

15000

In [6]:
def clean_text(txt):
  txt = "".join(v for v in txt if v not in string.punctuation).lower()
  txt = txt.encode("utf8").decode("ascii",'ignore')
  return txt 

In [7]:
corpus = [clean_text(x) for x in df_sub]
corpus[:10] 

[' i know  i was listenin to bad habit earlier and i started freakin at his part ',
 'layin n bed with a headache  ughhhhwaitin on your call',
 'funeral ceremonygloomy friday',
 'wants to hang out with friends soon',
 ' we want to trade with someone who has houston tickets but no one will',
 'repinging  why didnt you go to prom bc my bf didnt like my friends',
 'i should be sleep but im not thinking about an old friend who i want but hes married now   he wants me 2 scandalous',
 'hmmm httpwwwdjherocom is down',
 ' charlene my love i miss you',
 ' im sorry  at least its friday']

In [8]:
type(corpus)

list

#Generate Seq of n-grams

In [9]:
vocab = []
for line in corpus:
    words = line.split()
    for word in words:
        vocab.append(word)

vocabraly = set(vocab)

In [10]:
len(vocabraly)

19034

In [11]:
tokenizer = Tokenizer(2000)
tokenizer.fit_on_texts(corpus)
word2index = tokenizer.word_index
len(word2index)

19034

In [12]:
word2index

{'i': 1,
 'to': 2,
 'the': 3,
 'my': 4,
 'a': 5,
 'and': 6,
 'is': 7,
 'it': 8,
 'in': 9,
 'im': 10,
 'for': 11,
 'you': 12,
 'me': 13,
 'of': 14,
 'so': 15,
 'but': 16,
 'have': 17,
 'on': 18,
 'that': 19,
 'not': 20,
 'its': 21,
 'just': 22,
 'be': 23,
 'at': 24,
 'was': 25,
 'no': 26,
 'this': 27,
 'get': 28,
 'with': 29,
 'work': 30,
 'now': 31,
 'up': 32,
 'dont': 33,
 'out': 34,
 'go': 35,
 'cant': 36,
 'like': 37,
 'all': 38,
 'got': 39,
 'today': 40,
 'day': 41,
 'too': 42,
 'do': 43,
 'are': 44,
 'going': 45,
 'really': 46,
 'one': 47,
 'good': 48,
 'miss': 49,
 'back': 50,
 'know': 51,
 'want': 52,
 'from': 53,
 'what': 54,
 'time': 55,
 'am': 56,
 'sad': 57,
 'still': 58,
 'will': 59,
 'about': 60,
 'had': 61,
 'think': 62,
 'when': 63,
 'your': 64,
 'last': 65,
 '2': 66,
 'they': 67,
 'oh': 68,
 'we': 69,
 'home': 70,
 'there': 71,
 'u': 72,
 'feel': 73,
 'sorry': 74,
 'need': 75,
 'wish': 76,
 'bad': 77,
 'has': 78,
 'here': 79,
 'see': 80,
 'why': 81,
 'how': 82,
 'didnt'

In [13]:
dictionary = {}
rev_dictionary = {}
for word, idx in word2index.items():
    if idx > 1406:
        continue
    dictionary[word] = idx
    rev_dictionary[idx] = word

In [14]:
max(rev_dictionary.keys())

1406

In [15]:
input_seqences = tokenizer.texts_to_sequences(corpus)

In [16]:
input_seqences[:10]

[[1, 51, 1, 25, 2, 77, 623, 6, 1, 551, 810, 24, 155, 501],
 [256, 202, 29, 5, 258, 18, 64, 295],
 [1486, 161],
 [292, 2, 611, 34, 29, 198, 183],
 [69, 52, 2, 29, 190, 214, 78, 545, 16, 26, 47, 59],
 [81, 83, 12, 35, 2, 1003, 574, 4, 1574, 83, 37, 4, 198],
 [1,
  142,
  23,
  118,
  16,
  10,
  20,
  372,
  60,
  91,
  229,
  269,
  214,
  1,
  52,
  16,
  302,
  1388,
  31,
  99,
  292,
  13,
  66],
 [1047, 7, 138],
 [4, 100, 1, 49, 12],
 [10, 74, 24, 330, 21, 161]]

In [17]:
len(input_seqences)

15000

#Padding the Sequences and obtain Variables

In [18]:
input_data = []
target = []
for line in input_seqences:
    for i in range(1, len(line)-1):
        input_data.append(line[:i])
        target.append(line[i+1])

In [19]:
input_data[:5]

[[1], [1, 51], [1, 51, 1], [1, 51, 1, 25], [1, 51, 1, 25, 2]]

In [20]:
target[:5]

[1, 25, 2, 77, 623]

In [21]:
MAX_LEN = 0
for seq in input_data:
    if len(seq) > MAX_LEN:
        MAX_LEN = len(seq)
MAX_LEN

29

In [22]:
input_data = pad_sequences(input_data, maxlen=MAX_LEN, padding="post", truncating="post")

In [23]:
len(input_data[0])

29

In [24]:
input_data.shape

(140459, 29)

In [25]:
total_words = len(tokenizer.word_index) + 1

In [26]:
target = ku.np_utils.to_categorical(target, num_classes=total_words)

In [27]:
target

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [28]:
VOCAB_SIZE = 2001
VOCAB_SIZE

2001

In [ ]:
#LSTM For Text Gen
model = Sequential()
model.add(Embedding(input_dim=VOCAB_SIZE, output_dim=100, input_length=MAX_LEN))
model.add(LSTM(units=100))
model.add(Dropout(rate=0.1))
model.add(Dense(units=target.shape[1], activation="softmax"))

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer="adam")

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 29, 100)           200100    
                                                                 
 lstm (LSTM)                 (None, 100)               80400     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 14787)             1493487   
                                                                 
Total params: 1,773,987
Trainable params: 1,773,987
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(input_data, target, batch_size=32, epochs=5, verbose=1)

Epoch 1/5
2933/2933 [==============================] - 145s 48ms/step - loss: 6.2093
Epoch 2/5
2933/2933 [==============================] - 137s 47ms/step - loss: 6.0680
Epoch 3/5
2933/2933 [==============================] - 141s 48ms/step - loss: 6.0596
Epoch 4/5
2933/2933 [==============================] - 139s 47ms/step - loss: 6.0353
Epoch 5/5
2933/2933 [==============================] - 137s 47ms/step - loss: 5.9949


#GRU Model

In [29]:
gru_model = Sequential()
gru_model.add(Embedding(input_dim=VOCAB_SIZE, output_dim=100, input_length=MAX_LEN))
gru_model.add(GRU(units=100))
gru_model.add(Dropout(rate=0.1))
gru_model.add(Dense(units=target.shape[1], activation="softmax"))

In [30]:
gru_model.compile(loss="categorical_crossentropy", optimizer="adam")

In [31]:
gru_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 29, 100)           200100    
                                                                 
 gru (GRU)                   (None, 100)               60600     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 19035)             1922535   
                                                                 
Total params: 2,183,235
Trainable params: 2,183,235
Non-trainable params: 0
_________________________________________________________________


In [32]:
gru_model.fit(input_data, target, batch_size=32, epochs=5, verbose=1)

Epoch 1/5
4390/4390 [==============================] - 283s 64ms/step - loss: 6.1696
Epoch 2/5
4390/4390 [==============================] - 265s 60ms/step - loss: 5.9879
Epoch 3/5
4390/4390 [==============================] - 265s 60ms/step - loss: 5.8940
Epoch 4/5
4390/4390 [==============================] - 263s 60ms/step - loss: 5.8275
Epoch 5/5
4390/4390 [==============================] - 253s 58ms/step - loss: 5.7648


#Text Generation

In [33]:
from numpy.random import seed
import tensorflow
tensorflow.random.set_seed(2)
seed(1)

In [34]:
def text_generater(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len, padding="post")
        predicted = gru_model.predict(token_list, verbose=0)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted.all():
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

In [35]:
text1 = "trade with"
text_generater(text1, 5, gru_model, MAX_LEN)

'Trade I I'